<a href="https://colab.research.google.com/github/danielbauer1979/ML_656/blob/main/Assignment1_LinReg_Sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 1 -- Solution

Load packages:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

We rely on `statsmodels.api` here rather than scikit-learn, which is our general go-to tool. The reason is to illustrate a more "comfortable" way to linear regressions that has more of the feel and look of `R`.

Let's load our data:

In [ ]:
!git clone https://github.com/danielbauer1979/ML_656.git

In [ ]:
data = pd.read_csv('ML_656/tel.csv')
data.head()

Let's look at aggregate statistics:

In [ ]:
data.describe()

Having Day as a number doesn't make sense, so let's put in dummies. We will use Monday as our base and use dummies for Tuesday till Friday. (It's not clear whether it makes sense to use RWT and other varibles as numeric, but let's run with it for now.)

In [ ]:
data['Tuesday'] = data.apply(lambda row: int(row.Day==2), axis=1)
data['Wednesday'] = data.apply(lambda row: int(row.Day==3), axis=1)
data['Thursday'] = data.apply(lambda row: int(row.Day==4), axis=1)
data['Friday'] = data.apply(lambda row: int(row.Day==5), axis=1)
data.head()

Let's start running regressions, and let's start with the "full" model.

In [ ]:
# Assign dependent and independent / explanatory variables
y = data['Hours']
X = data.drop(columns=['Hours','Day'])
X = sm.add_constant(X) # Add a constant term as the default model doesn't include one
model = sm.OLS(y, X).fit()
# Check regression results
model.summary()

So it looks like only Friday really matters, so let's drop the other days.

In [ ]:
X = data.drop(columns=['Hours','Day','Tuesday','Wednesday','Thursday'])
X = sm.add_constant(X)
model2 = sm.OLS(y, X).fit()
model2.summary()

Let's also look at a reduced model with just three variables that seems significantly different than zero (their confidence interval does not include zero).

In [ ]:
X = data[['SOA','Hot','Friday']]
X = sm.add_constant(X)
model3 = sm.OLS(y, X).fit()
model3.summary()

In [ ]:
# ANOVA
from statsmodels.stats.api import anova_lm
anova_lm(model2, model3)

In [ ]:
# Check the residual
ypred = model3.predict(X)
plt.scatter(range(len(data)), y-ypred)
plt.show()

In [ ]:
plt.hist(y-ypred)
plt.show()